In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import json, pickle
from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, LayerGroup
# if above module isn't installed, do BOTH of the following:
# pip install ipyleaflet
# jupyter nbextension enable --py --sys-prefix ipyleaflet

from ipywidgets import HTML

In [ ]:
# Read in scraped car data
data_file = os.path.join(os.getcwd(),"..","data","all_cars.csv")  
cars = pd.read_csv(data_file)
pd.set_option("display.max_rows",None,"display.max_columns",None) 

# Recast data
cars['mileage'] = cars['mileage'].astype('float')
cars['year'] = cars['year'].astype('float')

# Clean price data and recast
ugly_cars = cars[cars['price'].str.contains('MSRP')]
ugly_cars.index
for index, car in ugly_cars.iterrows():
    if '|' not in car['price']:
        cars.at[index,'price'] = None
    else:
        cars.at[index,'price'] = car['price'].split('|')[0].strip()
cars['price'] = cars['price'].astype('float')

In [ ]:
# Data clean up
# Check year
# any cars with a year less than 1920 changed to None
cars.loc[(cars.year < 1920),'year']=None 

# Check price
# any cars with a price less than $100 or greater than $500,000 changed to None
cars.loc[(cars.price < 100),'price']=None 
cars.loc[(cars.price > 500000),'price']=None 

# Check Mileage
# any cars with a mileage greater than 500000 changed to None
cars.loc[(cars.mileage > 500000),'mileage']=None 

# Check Liters
# any cars with a mileage greater than 500000 changed to None
cars.loc[(cars.liters > 100),'liters']=None

# Check fav_per_view
# any cars with inf or Nan changed to None
cars.loc[(cars.fav_per_view > 2),'fav_per_view']=None 

In [ ]:
# Add in a city, state column
cars['citystate_abb'] = cars[['city', 'state']].apply(lambda x: ', '.join(x.astype(str)), axis=1)
# Combine make and model for new column
cars['make_model'] = cars[['make', 'model']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# only get non-null expected price rows
good_cars = cars[cars.expected_price.notnull()]

In [ ]:
# load zip code coordinates
with open('zip_coord_api.pkl','rb') as handle:
    zip_coord = pickle.load(handle)

In [ ]:
# reverse the ZIP:lat/long dictionary
coor_zip = {str(round(v[0], 3))+str(round(v[1], 3)): k for k, v in zip_coord.items()}

if len(coor_zip) != len(zip_coord):
    raise ValueError('the reversed ZIP code dictionary had a different number of keys than the original')

In [ ]:
good_cars = good_cars.copy()
good_cars['price_diff'] =  good_cars['price'] / good_cars['expected_price']
good_cars['price_cat'] = pd.qcut(good_cars['price_diff'], 3, labels=["green", "orange", "red"])
good_cars['price_abs_cat'] = pd.qcut(good_cars['price'], 3, labels=["green", "orange", "red"])
good_cars['price_label'] = pd.qcut(good_cars['price_diff'], 3, labels=["Good", "Average", "Bad"])
good_cars['price_abs_label'] = pd.qcut(good_cars['price'], 3, labels=["Cheap", "Average", "Expensive"])

good_cars['zip_code'] = good_cars['zip_code'].astype(int)
zip_group = good_cars.groupby('zip_code')

zip_group_mode = zip_group.agg(pd.Series.mode)
zip_group_median = zip_group.agg(pd.Series.median)
zip_group_count = zip_group.agg('count').iloc[:,0]
zip_group_count_log = np.floor(np.log(zip_group_count)+2)


In [ ]:
from uszipcode import SearchEngine

search = SearchEngine()

utah_center = [39.3210, -111.0937]
zoom = 6
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=utah_center, zoom=zoom)
m.add_control(FullScreenControl())

def hover_on(event, type, coordinates):
    # use coordinates to reverse look up data associated with those coordinates
    try:
        currzip = coor_zip[str(round(coordinates[0], 3))+str(round(coordinates[1], 3))]
    except:
        currzip = int(search.by_coordinates(40.4, -112, radius=30, returns=1)[0].zipcode)
    currdata = good_cars.loc[good_cars['zip_code'] == currzip]
    currlabel = zip_group_mode.loc[currzip, 'citystate_abb']
    currtotal = "{:,}".format(zip_group_count.loc[currzip])
    currmedprice = "{:,}".format(int(zip_group_median.loc[currzip, 'price']))
    currmedmile = "{:,}".format(int(zip_group_median.loc[currzip, 'mileage']))
    currmedyear = str(int(zip_group_median.loc[currzip, 'year']))
    if isinstance(zip_group_mode.loc[currzip, 'make_model'], str):
        currcommcar = zip_group_mode.loc[currzip, 'make_model']
        mid_msg = '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>'
    else:
        currcommcar = zip_group_mode.loc[currzip, 'make_model'][0]
        mid_msg = '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>'
        for currcommcar in zip_group_mode.loc[currzip, 'make_model'][1:]:
            mid_msg += '<tr><td>&emsp;</td><td>' + currcommcar + '</td></tr>'
    if isinstance(zip_group_mode.loc[currzip, 'price_label'], str):
        currdeal = zip_group_mode.loc[currzip, 'price_label']
    else:
        currdeal = zip_group_mode.loc[currzip, 'price_label'][0]
    
    # remove old popup layer
    if isinstance(m.layers[-1], Popup):
        m.remove_layer(m.layers[-1])

    # add a popup layer on hover over a city
    message = HTML()
    
    upper_msg = ('<h4><strong>' + currlabel + '</strong></h4>' +
                     '<table>' +
                         '<tr><td>Total Cars:&emsp;</td><td>' + currtotal + '</td></tr>' +
                         '<tr><td>Median Price:&emsp;</td><td>$' + currmedprice + '</td></tr>' +
                         '<tr><td>Median Mileage:&emsp;</td><td>' + currmedmile + '</td></tr>' +
                         '<tr><td>Median Year:&emsp;</td><td>' + currmedyear + '</td></tr>')
    lower_msg = ('<tr><td>Overall Market:&emsp;</td><td>' + currdeal + '</td></tr>' +
                     '</table>')

    message.value = upper_msg + mid_msg + lower_msg
    
    popup = Popup(location=coordinates, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m.add_layer(popup) # add the new layer

# create a layer group 
layer_group = LayerGroup()
for zipp, coord in zip_coord.items():
    circle = CircleMarker()
    circle.location = coord
    circle.radius = int(zip_group_count_log[zipp])
    circle.weight = 2
    circle.opacity = 0.8
    if isinstance(zip_group_mode.loc[zipp,'price_cat'], str):
        color = zip_group_mode.loc[zipp,'price_cat']
    else:
        color = zip_group_mode.loc[zipp,'price_cat'][0]
    circle.color = color
    circle.fill_color = color
    circle.fill_opacity = 0.3
    layer_group.add_layer(circle)
    circle.on_click(hover_on)

    

m.add_layer(layer_group)
m

In [ ]:
utah_center = [39.3210, -111.0937]
zoom = 6
m2 = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=utah_center, zoom=zoom)
m2.add_control(FullScreenControl())

def hover_on(event, type, coordinates):
    # use coordinates to reverse look up data associated with those coordinates
    try:
        currzip = coor_zip[str(round(coordinates[0], 3))+str(round(coordinates[1], 3))]
    except:
        currzip = int(search.by_coordinates(40.4, -112, radius=30, returns=1)[0].zipcode)
    currdata = good_cars.loc[good_cars['zip_code'] == currzip]
    currlabel = zip_group_mode.loc[currzip, 'citystate_abb']
    currtotal = "{:,}".format(zip_group_count.loc[currzip])
    currmedprice = "{:,}".format(int(zip_group_median.loc[currzip, 'price']))
    currmedmile = "{:,}".format(int(zip_group_median.loc[currzip, 'mileage']))
    currmedyear = str(int(zip_group_median.loc[currzip, 'year']))
    if isinstance(zip_group_mode.loc[currzip, 'make_model'], str):
        currcommcar = zip_group_mode.loc[currzip, 'make_model']
        mid_msg = '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>'
    else:
        currcommcar = zip_group_mode.loc[currzip, 'make_model'][0]
        mid_msg = '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>'
        for currcommcar in zip_group_mode.loc[currzip, 'make_model'][1:]:
            mid_msg += '<tr><td>&emsp;</td><td>' + currcommcar + '</td></tr>'
    if isinstance(zip_group_mode.loc[currzip, 'price_abs_label'], str):
        currdeal = zip_group_mode.loc[currzip, 'price_abs_label']
    else:
        currdeal = zip_group_mode.loc[currzip, 'price_abs_label'][0]
        
    # remove old popup layer
    if isinstance(m2.layers[-1], Popup):
        m2.remove_layer(m2.layers[-1])

    # add a popup layer on hover over a city
    message = HTML()
    
    upper_msg = ('<h4><strong>' + currlabel + '</strong></h4>' +
                     '<table>' +
                         '<tr><td>Total Cars:&emsp;</td><td>' + currtotal + '</td></tr>' +
                         '<tr><td>Median Price:&emsp;</td><td>$' + currmedprice + '</td></tr>' +
                         '<tr><td>Median Mileage:&emsp;</td><td>' + currmedmile + '</td></tr>' +
                         '<tr><td>Median Year:&emsp;</td><td>' + currmedyear + '</td></tr>')
    lower_msg = ('<tr><td>Overall Market:&emsp;</td><td>' + currdeal + '</td></tr>' +
                     '</table>')

    message.value = upper_msg + mid_msg + lower_msg
    
    popup = Popup(location=coordinates, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m2.add_layer(popup) # add the new layer

# create a layer group 
layer_group = LayerGroup()
for zipp, coord in zip_coord.items():
    circle = CircleMarker()
    circle.location = coord
    circle.radius = int(zip_group_count_log[zipp])
    circle.weight = 2
    circle.opacity = 0.8
    if isinstance(zip_group_mode.loc[zipp,'price_abs_cat'], str):
        color = zip_group_mode.loc[zipp,'price_abs_cat']
    else:
        color = zip_group_mode.loc[zipp,'price_abs_cat'][0]
    circle.color = color
    circle.fill_color = color
    circle.fill_opacity = 0.3
    layer_group.add_layer(circle)
    circle.on_click(hover_on)

    

m2.add_layer(layer_group)
m2